0. Set Up

In [ ]:
!git clone --branch feature/002-rotation https://github.com/antonfrancois/Demeter_metamorphosis.git

Install packages

In [ ]:
import subprocess
import sys, os

cwd = subprocess.check_output("pwd", text=True).strip()
if not "Demeter" in cwd:
	%cd Demeter_metamorphosis/

!pip install pynrrd
!pip install -U ipywidgets
print("\nInstalation of demeter from pip")
!pip install -e .
# !pip install -e .

repo_path = os.path.abspath('.')
# repo_path = "/content/Demeter_metamorphosis/"
if repo_path not in sys.path:
	sys.path.insert(0, repo_path)
print(sys.path)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

0.2 After jupyter restart, run:

In [ ]:
%matplotlib widget

import sys
import os


# Remplace par le chemin relatif où se trouve le package 'demeter'
# Exemple : si ton notebook est dans le dossier root du repo, et demeter est un sous-dossier
import subprocess
cwd = subprocess.check_output("pwd", text=True).strip()
if not "Demeter" in cwd:
	%cd Demeter_metamorphosis/

# repo_path = os.path.abspath('.')
repo_path = "/content/Demeter_metamorphosis/"
if repo_path not in sys.path:
	sys.path.insert(0, repo_path)
print(sys.path)



# 1. Code utile

In [ ]:
import os
import nibabel as nib
import numpy as np
import torch

import demeter.utils.torchbox as tb
import demeter.utils.axes3dsliders_plt as a3s
import demeter.utils.rigid_exploration as rg
## Faire Recalage Rigid + Meta
import demeter.metamorphosis as mt
import demeter.metamorphosis.rotate as mtrt
import demeter.utils.reproducing_kernels as rk
import matplotlib.pyplot as plt


def to_torch(img):
    return torch.from_numpy(img)[None, None]

def normalize(img):
    quant = np.quantile(img, 0.99)
    print(quant, img.max())
    img = np.clip(img, 0,quant)
    img /= img.max()
    return img

## Open images

In [ ]:
# Open template
template_folder ="/home/turtlefox/Documents/11_metamorphoses/data/templates/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c"
template_name = "mni_icbm152_t1_tal_nlin_asym_09c.nii"
template_mask_name = "mni_icbm152_t1_tal_nlin_asym_09c_mask.nii"
template = nib.load(os.path.join(template_folder, template_name)).get_fdata()
template_mask = nib.load(os.path.join(template_folder, template_mask_name)).get_fdata()
template[template_mask == 0] = 0
# TODO : Open segs

print("template :",template.shape)

# Open ixi image
ixi_folder = "/home/turtlefox/Documents/11_metamorphoses/data/IXI-T1_fastsurfer/IXI002/mri/"
# ixi_list = os.listdir(ixi_folder)
ixi_image_name = "orig_nu.mgz"
ixi_segs_name = "aseg.auto_noCCseg.mgz"
ixi_mask_name = "mask.mgz"
ixi_img = nib.load(os.path.join(ixi_folder, ixi_image_name)).get_fdata()
ixi_segs = nib.load(os.path.join(ixi_folder, ixi_segs_name)).get_fdata()
ixi_mask = nib.load(os.path.join(ixi_folder, ixi_mask_name)).get_fdata()
ixi_img[ixi_mask == 0] = 0
# ixi_img= ixi_img[:, ::-1]
print("ixi_img :",ixi_img.shape)
print("ixi_segs :",ixi_segs.shape)
name = "IXI002_to_template"


# Normalise images to [0,1]
template = normalize(template)
ixi_img = normalize(ixi_img)

#  Resize ixi image
template = to_torch(template)
template = tb.resize_image(template, scale_factor= 1)
ixi_img = to_torch(ixi_img)
ixi_img = tb.resize_image(ixi_img, to_shape=template.shape[2:])

print(ixi_img.shape)

## Recherche rigide


In [ ]:
# Recherche rigide
source_b, target_b, trans_s, trans_t = rg.align_barycentres(ixi_img, template, verbose=True)

kernelOperator = rk.GaussianRKHS(sigma=(15,15,15),normalized=False)
datacost = mt.Rotation_Ssd_Cost(target_b.to('cuda:0'), alpha=1)
# datacost = mt.Rotation_Cost(target_b, mt.Mutual_Information, alpha=1)

mr = mt.rigid_along_metamorphosis(
    source_b, target_b, momenta_ini=0,
    kernelOperator= kernelOperator,
    rho = 1,
    data_term=datacost ,
    integration_steps = 10,
    cost_cst=.1,
)
top_params = rg.initial_exploration(mr,r_step=10, max_output = 15, verbose=True)
print(top_params)
best_loss, best_momentum_R, best_momentum_T, best_rot = rg.optimize_on_rigid(mr, top_params, n_iter=10,verbose=True)


## Check rigid search

In [ ]:
momenta = mtrt.prepare_momenta(
    # mr.source.shape,
    source_b.shape,
    image=False,
    rotation=True,
    translation=True,
    rot_prior=best_momentum_R.detach().clone(),trans_prior=best_momentum_T.detach().clone(),# - trans_s + trans_t,
    device= "cpu"
)
print(momenta["momentum_R"])
print(momenta["momentum_T"])
kernelOperator = rk.GaussianRKHS(sigma=(10,10,10),normalized=False)
datacost = mt.Rotation_Ssd_Cost(target_b.to('cuda:0'), alpha=.5)
# datacost = mt.Rotation_Cost(target_b, mt.Mutual_Information, alpha=.5)
mr = mt.rigid_along_metamorphosis(
    source_b, target_b, momenta_ini=momenta,
    kernelOperator= kernelOperator,
    rho = 1,
    data_term=datacost ,
    integration_steps = 10,
    cost_cst=.1,
    n_iter=0,
    save_gpu_memory=False,
    lbfgs_max_iter = 20,
    lbfgs_history_size = 20,
)
mr.mp.forward(source_b.cpu(), momenta, save=True)

print(mr.mp.rot_mat)

# a3s.Visualize_GeodesicOptim_plt(mr, "random")
# rot_def = tb.grid_from_rotation_translation(mr.mp.id_grid, mr.mp.rot_mat, mr.mp.translation)
# rot_def =   tb.grid_from_rotation(mr.mp.id_grid, mr.mp.rot_mat.T)
# rot_def += mr.mp.translation
rot_def = mr.mp.get_rigidor()
rotated_source = tb.imgDeform(mr.mp.image,rot_def,dx_convention='2square')
img = rotated_source[0,0,..., mr.source.shape[-1]//2].detach().cpu()
img_target = tb.imCmp(rotated_source[..., ixi_img.shape[-1]//2].detach().cpu(), mr.target[..., ixi_img.shape[-1]//2].detach().cpu(), "compose")[0]
img_source = tb.imCmp(rotated_source[..., ixi_img.shape[-1]//2].detach().cpu(), mr.source[..., ixi_img.shape[-1]//2].detach().cpu(), "compose")[0]
fig,ax = plt.subplots(1,3)
ax[0].imshow(img, cmap="gray")
ax[0].set_title("Final image")
ax[1].imshow(img_target, cmap="gray")
ax[1].set_title("img vs target")
ax[2].imshow(img_source, cmap="gray")
ax[2].set_title("img vs source")
plt.show()

## LDDMM

In [ ]:

sigma= [3, 5, 7]
sigma = [(s,)*3 for s in sigma]
kernelOperator = rk.Multi_scale_GaussianRKHS(sigma, normalized=False)

datacost = mt.Rotation_Ssd_Cost(target_b.to("cuda:0"), alpha=.5)
# datacost = mt.Rotation_MutualInformation_Cost(target_b, alpha=.5)

momenta = mtrt.prepare_momenta(
    # mr.source.shape,
    ixi_img.shape,
    image=True,
    rotation=True,
    translation=True,
    rot_prior=best_momentum_R.detach().clone(),trans_prior=best_momentum_T.detach().clone(),
)

for v in momenta.values():
    print(v.requires_grad)
rho = 1
mr = mt.rigid_along_metamorphosis(
    source_b, target_b, momenta_ini=momenta,
    kernelOperator= kernelOperator,
    rho = rho,
    data_term=datacost ,
    integration_steps = 10,
    cost_cst=1e-6,
    n_iter=15,
    save_gpu_memory=False,
    lbfgs_max_iter = 20,
    lbfgs_history_size = 20,
)


mr.plot_cost()


In [ ]:
# fig, ax = plt.subplots(1,3)
T, _, D, H, W = ixi_img.shape
print(f"residual min {mr.mp.residuals.min()} max {mr.mp.residuals.max()}")
# Choose a central slice for plotting
slice_index = W // 2
ax[0].imshow(mr.mp.residuals[0,0,..., slice_index].cpu(), cmap='gray')
ax[0].set_title(f'residuals')
ax[1].imshow(mr.mp.image[0,0,..., slice_index].cpu(), cmap='gray')
ax[1].set_title(f'image')

ax[2].imshow(mr.mp.target[0,0,..., slice_index], cmap='gray')
ax[2].set_title(f'target')
plt.show()

In [ ]:
a3s.Visualize_GeodesicOptim_plt(mr, f"{name}_metaRigid_rho{rho}")


In [ ]:
mr.save(f"{name}_rigid_metamorphosis_colab",save_path="/content/drive/MyDrive/demeter_data/demeter_optim/")
